# Aggregate Development and Population

### Setup

In [5]:
# import packages
import arcpy
import os
import pyodbc
import pandas as pd
import numpy as np
from sqlalchemy.engine import URL, create_engine
from arcgis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor

from arcgis.features import FeatureLayer

# set overwrite to true
arcpy.env.overwriteOutput = True

# enterprise Geodatabase connection
sdeBase = "F:\GIS\DB_CONNECT\Vector.sde"

# setup sql connection
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde;UID=sde;PWD=staff"
connection_url    = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
connection        = create_engine(connection_url)

# set workspace
arcpy.env.workspace = "F:\GIS\PROJECTS\ResearchAnalysis\Demographics\Workspace.gdb"
workspace           = r"F:\GIS\PROJECTS\ResearchAnalysis\Demographics\Workspace.gdb"
workspace_folder    = r"F:\GIS\PROJECTS\ResearchAnalysis\Demographics"

# in memory output file path
memory_workspace = "memory" + "\\"

# Connect to TRPA Enterprise GIS Portal
portal_url = "https://maps.trpa.org/portal"
gis = GIS(portal_url)

## Get Data

### Get Data from Database

In [2]:
# sde census features
censusGeography = os.path.join(sdeBase, 
                    "sde.SDE.Census\sde.SDE.Tahoe_Census_Geography")

# make feature layer of blocks from 2020
blockLayer2020 = arcpy.MakeFeatureLayer_management(
                        censusGeography, 
                        "Tahoe_Blocks_2020", 
                        "YEAR = 2020 And GEOGRAPHY = 'block'")

# make feature layer of blocks from 2010
blockLayer2010 = arcpy.MakeFeatureLayer_management(
                        censusGeography, 
                        "Tahoe_Blocks_2010", 
                        "YEAR = 2010 And GEOGRAPHY = 'block'")


# export copy to workspace
arcpy.conversion.FeatureClassToGeodatabase(blockLayer2020,workspace)

# export copy to workspace
arcpy.conversion.FeatureClassToGeodatabase(blockLayer2010,workspace)

# field list for new block feature class
field_list_2010 = [
 # Add fields to block group 2010
 'Block_Population_2010',
 'Block_HousingUnits_2010',
 'Block_Occupied_2010',
 'Block_Vacant_2010',
 'Block_Seasonal_2010',
 'Block_ResidenatialUnits_2010',
]

field_list_2020=[
# Add fields to block group 2020
 'Block_Population_2020',
 'Block_HousingUnits_2020',
 'Block_Occupied_2020',
 'Block_Vacant_2020',
 'Block_Seasonal_2020',
 'Block_ResidenatialUnits_2020',
 # Add fields for % change
 'Block_PercentChange_Population',
 'Block_PercentChange_HousingUnits',
 'Block_PercentChange_ResidentialUnits',
]

# get at workspace blocks
tahoeBlocks2010 = os.path.join(workspace, 'Tahoe_Blocks_2010')
# get at workspace blocks
tahoeBlocks2020 = os.path.join(workspace, 'Tahoe_Blocks_2020')

# add fields to workspace blocks
for field in field_list_2010:
    arcpy.AddField_management(tahoeBlocks2010, field, "DOUBLE")

for field in field_list_2020:
    arcpy.AddField_management(tahoeBlocks2020, field, "DOUBLE")

### Get data from feature service

In [3]:
# Search for the feature service by keyword
feature_layer_item = gis.content.search(query="Demographics", item_type="Feature Layer")[0]

# feature sub layer name to get
sublayer_name = "Tahoe Census Geography"

# Query the sublayer by name
sublayer = None
for layer in feature_layer_item.layers:
    if layer.properties.name == sublayer_name:
        sublayer = layer
        break

# create a Spatially Enabled DataFrame
sdfCensus = pd.DataFrame.spatial.from_layer(sublayer)

# create data frames from filter for block and year
sdfBlocks2020 = sdfCensus[(sdfCensus['GEOGRAPHY'] == 'block') & (sdfCensus['YEAR'] == 2020)]
sdfBlocks2010 = sdfCensus[(sdfCensus['GEOGRAPHY'] == 'block') & (sdfCensus['YEAR'] == 2010)]

In [9]:
# Search for the feature layer by keyword
feature_layer_item = gis.content.search(query="Demographics", item_type="Feature Layer")[0]

# # Access the first result (assuming it's the one you want)
# feature_layer_item = search_result[0]
table_name = "Census Data"

# Query the sublayer by name
subtable = None
for table in feature_layer_item.tables:
    if table.properties.name == table_name:
        subtable = table
        break

# create a Spatially Enabled DataFrame object
dfCensus = pd.DataFrame.spatial.from_layer(subtable)

# Define the filter conditions for each field
conditionBlock      = dfCensus['sample_level']  == 'block'
condition2010       = dfCensus['year_sample']   == 2010
condition2020       = dfCensus['year_sample']   == 2020
conditionPopulation = dfCensus['variable_name'] == 'Total Population'
conditionHousing    = dfCensus['variable_name'] == 'Total Housing Units'
conditionOccupied   = dfCensus['variable_name'] == 'Total Housing Units: Occupied'
conditionVacant     = dfCensus['variable_name'] == 'Total Housing Units: Vacant'
conditionSeasonal   = dfCensus['variable_name'] == 'Vacant Housing Units: Seasonal, recreational, or occasional use'

# filter to create new dfs by variable name
dfBlockPop2010           =  dfCensus.loc[conditionBlock & condition2010 & conditionPopulation].copy()
dfBlockUnits2010         =  dfCensus.loc[conditionBlock & condition2010 & conditionHousing].copy()
dfBlockUnitsOccupied2010 =  dfCensus.loc[conditionBlock & condition2010 & conditionOccupied].copy()
dfBlockUnitsVacant2010   =  dfCensus.loc[conditionBlock & condition2010 & conditionVacant].copy()
dfBlockUnitsSeasonal2010 =  dfCensus.loc[conditionBlock & condition2010 & conditionSeasonal].copy()

# create 2020 data frames from sql query
dfBlockPop2020           =  dfCensus[conditionBlock & condition2020 & conditionPopulation].copy()
dfBlockUnits2020         =  dfCensus[conditionBlock & condition2020 & conditionHousing].copy()
dfBlockUnitsOccupied2020 =  dfCensus[conditionBlock & condition2020 & conditionOccupied].copy()
dfBlockUnitsVacant2020   =  dfCensus[conditionBlock & condition2020 & conditionVacant].copy()
dfBlockUnitsSeasonal2020 =  dfCensus[conditionBlock & condition2020 & conditionSeasonal].copy()

## Join Data Frames and Blocks

### Add Data Frame name as a prefix to column names

In [8]:
df_list = [
    dfBlockPop2010,          
    dfBlockUnits2010,         
    dfBlockUnitsOccupied2010, 
    dfBlockUnitsVacant2010,  
    dfBlockUnitsSeasonal2010,
    dfBlockPop2020,           
    dfBlockUnits2020,         
    dfBlockUnitsOccupied2020, 
    dfBlockUnitsVacant2020,
    dfBlockUnitsSeasonal2020
]

# specify fields to keep
fields_to_keep = ['TRPAID', 'value']

# Loop through each DataFrame and drop the specified fields
for dataframe in df_list:
    # drop columns not in list
    dataframe.drop(columns=[col for col in dataframe.columns if col not in fields_to_keep], inplace=True)  
    # Exclude the field name you want to skip
    field_to_exclude = "TRPAID"
    # keep neccessary columns
    included_columns = [col for col in dataframe.columns if col != field_to_exclude]
    # get the dataframe name as a string
    df_name = [name for name in globals() if globals()[name] is dataframe][0]
    dataframe['TRPAID'].astype(str)
    # Add DataFrame name as a prefix to included column names
    new_columns = [f"{df_name}_{col}" for col in included_columns]
    dataframe.columns =  ["TRPAID"]+ new_columns
    

### Merge data frames to spatial data frames

In [9]:
df_list_2010 = [
    dfBlockPop2010,          
    dfBlockUnits2010,         
    dfBlockUnitsOccupied2010, 
    dfBlockUnitsVacant2010,  
    dfBlockUnitsSeasonal2010
]

df_list_2020=[
    dfBlockPop2020,           
    dfBlockUnits2020,         
    dfBlockUnitsOccupied2020, 
    dfBlockUnitsVacant2020,
    dfBlockUnitsSeasonal2020
]
# Join DataFrames using merge
for df in df_list_2010:
    sdfBlocks2010 = sdfBlocks2010.merge(df, on='TRPAID', how="left")
# Join DataFrames using merge
for df in df_list_2020:
    sdfBlocks2020 = sdfBlocks2020.merge(df, on='TRPAID', how="left")
    
# set new field values
sdfBlocks2010['Block_Population_2010']   = sdfBlocks2010['dfBlockPop2010_value']
sdfBlocks2010['Block_HousingUnits_2010'] = sdfBlocks2010['dfBlockUnits2010_value']
sdfBlocks2010['Block_Occupied_2010']     = sdfBlocks2010['dfBlockUnitsOccupied2010_value']
sdfBlocks2010['Block_Vacant_2010']       = sdfBlocks2010['dfBlockUnitsVacant2010_value']
sdfBlocks2010['Block_Seasonal_2010']     = sdfBlocks2010['dfBlockUnitsSeasonal2010_value']

sdfBlocks2020['Block_Population_2020']   = sdfBlocks2020['dfBlockPop2020_value']
sdfBlocks2020['Block_HousingUnits_2020'] = sdfBlocks2020['dfBlockUnits2020_value']
sdfBlocks2020['Block_Occupied_2020']     = sdfBlocks2020['dfBlockUnitsOccupied2020_value']
sdfBlocks2020['Block_Vacant_2020']       = sdfBlocks2020['dfBlockUnitsVacant2020_value']
sdfBlocks2020['Block_Seasonal_2020']     = sdfBlocks2020['dfBlockUnitsSeasonal2020_value']

### Export to Data Frames to staging Feature Classes

In [21]:
df = sdfBlocks2010

numeric_columns = []

for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        numeric_columns.append(column)
        
# Fill specified numeric columns with zeros
df[numeric_columns] = df[numeric_columns].fillna(0)

## Export spatial dataframes to feature class to use in Spatial join
sdfBlocks2010.copy().spatial.to_featureclass(os.path.join(workspace, "Tahoe_Blocks_2010_Staging"), sanitize_columns=False)

df = sdfBlocks2020

numeric_columns = []

for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        numeric_columns.append(column)
        
# Fill specified numeric columns with zeros
df[numeric_columns] = df[numeric_columns].fillna(0)

## Export spatial dataframes to feature class to use in Spatial join
sdfBlocks2020.copy().spatial.to_featureclass(os.path.join(workspace, "Tahoe_Blocks_2020_Staging"), sanitize_columns=False)

'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\Tahoe_Blocks_2020_Staging'

### Spatial Joins of 2022 development and 2010 census block data

In [22]:
field_names = [f.name for f in arcpy.ListFields("Tahoe_Blocks_2010_Staging")]
print(field_names)

['OBJECTID', 'Shape', 'Shape_Length', 'Shape_Area', 'GEOGRAPHY', 'GEOID', 'GlobalID', 'NEIGHBORHOOD', 'STATE', 'Shape__Area', 'Shape__Length', 'TRPAID', 'YEAR', 'created_date', 'created_user', 'last_edited_date', 'last_edited_user', 'dfBlockPop2010_value', 'dfBlockUnits2010_value', 'dfBlockUnitsOccupied2010_value', 'dfBlockUnitsVacant2010_value', 'dfBlockUnitsSeasonal2010_value', 'Block_Population_2010', 'Block_HousingUnits_2010', 'Block_Occupied_2010', 'Block_Vacant_2010', 'Block_Seasonal_2010']


In [23]:
field_names = [f.name for f in arcpy.ListFields("Tahoe_Blocks_2020_Staging")]
print(field_names)

['OBJECTID', 'Shape', 'Shape_Length', 'Shape_Area', 'GEOGRAPHY', 'GEOID', 'GlobalID', 'NEIGHBORHOOD', 'STATE', 'Shape__Area', 'Shape__Length', 'TRPAID', 'YEAR', 'created_date', 'created_user', 'last_edited_date', 'last_edited_user', 'dfBlockPop2020_value', 'dfBlockUnits2020_value', 'dfBlockUnitsOccupied2020_value', 'dfBlockUnitsVacant2020_value', 'dfBlockUnitsSeasonal2020_value', 'Block_Population_2020', 'Block_HousingUnits_2020', 'Block_Occupied_2020', 'Block_Vacant_2020', 'Block_Seasonal_2020']


In [25]:
target_feature_class = os.path.join(workspace, "Tahoe_Blocks_2020_Staging")
join_feature_class   = os.path.join(workspace, "Tahoe_Blocks_2010_Staging")
output_feature_class = "SpatialJoin_Blocks2020_Blocks2010"

# List of input and join field names
input_field_names = [ 
                'GEOGRAPHY', 'GEOID', 'GlobalID', 'NEIGHBORHOOD', 'STATE', 'TRPAID', 'YEAR', 
                'Block_Population_2020', 'Block_HousingUnits_2020', 
                'Block_Occupied_2020', 'Block_Vacant_2020', 'Block_Seasonal_2020'
                    ]

join_field_names = [ 'YEAR',
                    'Block_Population_2010', 'Block_HousingUnits_2010', 
                    'Block_Occupied_2010', 'Block_Vacant_2010', 'Block_Seasonal_2010'
                   ]

# List of fields to be joined using SUM
fields_to_sum = [
                    'Block_Population_2010', 'Block_HousingUnits_2010', 
                    'Block_Occupied_2010', 'Block_Vacant_2010', 'Block_Seasonal_2010'
                ]

# Create a field map object
field_mappings = arcpy.FieldMappings()

# Get the field info for the target feature class
target_field_info = arcpy.ListFields(target_feature_class)

# Add fields from the target feature class to the field mappings
for field in target_field_info:
    if field.name in input_field_names:
        input_field_map = arcpy.FieldMap()
        input_field_map.addInputField(target_feature_class, field.name)
        # Set the merge rule to SUM for selected fields
        if field.name in fields_to_sum:
            input_field_map.mergeRule = "SUM"
        
        field_mappings.addFieldMap(input_field_map)

# Get the field info for the join feature class
join_field_info = arcpy.ListFields(join_feature_class)

# Add fields from the join feature class to the field mappings
for field in join_field_info:
    if field.name in join_field_names:
        join_field_map = arcpy.FieldMap()
        join_field_map.addInputField(join_feature_class, field.name)
        # Set the merge rule to SUM for selected fields
        if field.name in fields_to_sum:
            join_field_map.mergeRule = "SUM"
        # add join fields to field mappings
        field_mappings.addFieldMap(join_field_map)

# Perform the spatial join using the specified field mappings
arcpy.analysis.SpatialJoin(
                    target_feature_class, 
                    join_feature_class, 
                    output_feature_class,
                    join_operation="JOIN_ONE_TO_ONE",
                    join_type="KEEP_ALL",
                    field_mapping = field_mappings,
                    match_option="HAVE_THEIR_CENTER_IN",
                    search_radius=None,
                    distance_field_name="")


<Result 'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\SpatialJoin_Blocks2020_Blocks2010'>

### Spatial Join Development rights to Blocks

In [33]:
# Set the definition query
definition_query = "YEAR = 2022"

# Create a feature layer
parcel_development_2022 = arcpy.management.MakeFeatureLayer(
                            in_features=os.path.join(workspace, 'Parcel_Development'),
                            out_layer="Parcel Development 2022",
                            where_clause=definition_query
                            )[0]

field_names = [f.name for f in arcpy.ListFields(parcel_development_2022)]
print(field_names)

['OBJECTID', 'Shape', 'APN', 'PPNO', 'APO_ADDRESS', 'Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt', 'YEAR', 'OWN_FULL', 'JURISDICTION', 'COUNTY', 'OWNERSHIP_TYPE', 'COUNTY_LANDUSE_CODE', 'COUNTY_LANDUSE_DESCRIPTION', 'EXISTING_LANDUSE', 'REGIONAL_LANDUSE', 'AS_SUM', 'TAX_SUM', 'YEAR_BUILT', 'UNITS', 'BEDROOMS', 'BATHROOMS', 'BUILDING_SQFT', 'ESTIMATED_COVERAGE_ALLOWED', 'IMPERVIOUS_SURFACE_SQFT', 'PLAN_ID', 'PLAN_NAME', 'ZONING_ID', 'ZONING_DESCRIPTION', 'TOWN_CENTER', 'LOCATION_TO_TOWNCENTER', 'TAZ', 'WITHIN_TRPA_BNDY', 'PARCEL_ACRES', 'PARCEL_SQFT', 'WITHIN_BONUSUNIT_BNDY', 'Shape_Length', 'Shape_Area']


In [44]:
# Set the definition query
definition_query = "YEAR = 2012"

# Create a feature layer
parcel_development_2012 = arcpy.management.MakeFeatureLayer(
                            in_features=os.path.join(workspace, 'Parcel_Development'),
                            out_layer="Parcel Development 2012",
                            where_clause=definition_query
                            )[0]

# export to points to get a layer that will join
outFeatureClass = "ParcelDev2012_points"

# Use FeatureToPoint function to find a point inside each park
arcpy.management.FeatureToPoint(parcel_development_2012, outFeatureClass, "INSIDE")

# fields
field_names = [f.name for f in arcpy.ListFields(parcel_development_2012)]
print(field_names)

['OBJECTID', 'Shape', 'APN', 'PPNO', 'APO_ADDRESS', 'Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt', 'YEAR', 'OWN_FULL', 'JURISDICTION', 'COUNTY', 'OWNERSHIP_TYPE', 'COUNTY_LANDUSE_CODE', 'COUNTY_LANDUSE_DESCRIPTION', 'EXISTING_LANDUSE', 'REGIONAL_LANDUSE', 'AS_SUM', 'TAX_SUM', 'YEAR_BUILT', 'UNITS', 'BEDROOMS', 'BATHROOMS', 'BUILDING_SQFT', 'ESTIMATED_COVERAGE_ALLOWED', 'IMPERVIOUS_SURFACE_SQFT', 'PLAN_ID', 'PLAN_NAME', 'ZONING_ID', 'ZONING_DESCRIPTION', 'TOWN_CENTER', 'LOCATION_TO_TOWNCENTER', 'TAZ', 'WITHIN_TRPA_BNDY', 'PARCEL_ACRES', 'PARCEL_SQFT', 'WITHIN_BONUSUNIT_BNDY', 'Shape_Length', 'Shape_Area']


In [45]:
# Set the definition query
definition_query = "YEAR = 2022"

# Create a feature layer
parcel_development_2012 = arcpy.management.MakeFeatureLayer(
                            in_features=os.path.join(workspace, 'Parcel_Development'),
                            out_layer="Parcel Development 2022",
                            where_clause=definition_query
                            )[0]

# export to points to get a layer that will join
outFeatureClass = "ParcelDev2022_points"

# Use FeatureToPoint function to find a point inside each park
arcpy.management.FeatureToPoint(parcel_development_2012, outFeatureClass, "INSIDE")

# fields
field_names = [f.name for f in arcpy.ListFields(parcel_development_2012)]
print(field_names)

['OBJECTID', 'Shape', 'APN', 'PPNO', 'APO_ADDRESS', 'Residential_Units', 'TouristAccommodation_Units', 'CommercialFloorArea_SqFt', 'YEAR', 'OWN_FULL', 'JURISDICTION', 'COUNTY', 'OWNERSHIP_TYPE', 'COUNTY_LANDUSE_CODE', 'COUNTY_LANDUSE_DESCRIPTION', 'EXISTING_LANDUSE', 'REGIONAL_LANDUSE', 'AS_SUM', 'TAX_SUM', 'YEAR_BUILT', 'UNITS', 'BEDROOMS', 'BATHROOMS', 'BUILDING_SQFT', 'ESTIMATED_COVERAGE_ALLOWED', 'IMPERVIOUS_SURFACE_SQFT', 'PLAN_ID', 'PLAN_NAME', 'ZONING_ID', 'ZONING_DESCRIPTION', 'TOWN_CENTER', 'LOCATION_TO_TOWNCENTER', 'TAZ', 'WITHIN_TRPA_BNDY', 'PARCEL_ACRES', 'PARCEL_SQFT', 'WITHIN_BONUSUNIT_BNDY', 'Shape_Length', 'Shape_Area']


In [52]:
# Spatial join to parcel development
target_feature_class = os.path.join(workspace, "SpatialJoin_Blocks2020_Blocks2010")
join_feature_class   = "ParcelDev2012_points"
output_feature_class = "SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012"

# List of input and join field names
input_field_names = [ 
                'GEOGRAPHY', 'GEOID', 'GlobalID', 'NEIGHBORHOOD', 'STATE', 'TRPAID', 'YEAR', 
                'Block_Population_2020', 'Block_HousingUnits_2020', 
                'Block_Occupied_2020', 'Block_Vacant_2020', 'Block_Seasonal_2020',
                'Block_Population_2010', 'Block_HousingUnits_2010', 
                'Block_Occupied_2010', 'Block_Vacant_2010', 'Block_Seasonal_2010'
                    ]

join_field_names = [ 'Residential_Units'
                   ]

# List of fields to be joined using SUM
fields_to_sum = ['Residential_Units'
                ]

# Create a field map object
field_mappings = arcpy.FieldMappings()

# Get the field info for the target feature class
target_field_info = arcpy.ListFields(target_feature_class)

# Add fields from the target feature class to the field mappings
for field in target_field_info:
    if field.name in input_field_names:
        input_field_map = arcpy.FieldMap()
        input_field_map.addInputField(target_feature_class, field.name)
        # Set the merge rule to SUM for selected fields
        if field.name in fields_to_sum:
            input_field_map.mergeRule = "SUM"
        
        field_mappings.addFieldMap(input_field_map)

# Get the field info for the join feature class
join_field_info = arcpy.ListFields(join_feature_class)

# Add fields from the join feature class to the field mappings
for field in join_field_info:
    if field.name in join_field_names:
        join_field_map = arcpy.FieldMap()
        join_field_map.addInputField(join_feature_class, field.name)
        # Set the merge rule to SUM for selected fields
        if field.name in fields_to_sum:
            join_field_map.mergeRule = "SUM"
        # add join fields to field mappings
        field_mappings.addFieldMap(join_field_map)

# Perform the spatial join using the specified field mappings
arcpy.analysis.SpatialJoin(
                    target_feature_class, 
                    join_feature_class, 
                    output_feature_class,
                    join_operation="JOIN_ONE_TO_ONE",
                    join_type="KEEP_ALL",
                    field_mapping = field_mappings,
                    match_option="CONTAINS",
                    search_radius=None,
                    distance_field_name="")


<Result 'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012'>

In [53]:
# Set local variables
in_table = os.path.join(workspace,"SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012") 
field = "Residential_Units"
new_field_name = "Residential_Units_2012"
new_field_alias = "Residential Units 2012"
field_type = "LONG"

# Alter the properties of a non nullable, short data type field to become a text field
arcpy.management.AlterField(in_table,
                            field,
                            new_field_name,
                            new_field_alias,
                            field_type)

<Result 'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012'>

In [64]:
# Spatial join to parcel development
target_feature_class = os.path.join(workspace, "SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012")
join_feature_class   = "ParcelDev2022_points"
output_feature_class = "SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012_ParcelDev2022"

# List of input and join field names
input_field_names = [ 
                'GEOGRAPHY', 'GEOID', 'GlobalID', 'NEIGHBORHOOD', 'STATE', 'TRPAID', 'YEAR', 
                'Block_Population_2020', 'Block_HousingUnits_2020', 
                'Block_Occupied_2020', 'Block_Vacant_2020', 'Block_Seasonal_2020',
                'Block_Population_2010', 'Block_HousingUnits_2010', 
                'Block_Occupied_2010', 'Block_Vacant_2010', 'Block_Seasonal_2010', 'Residential_Units_2012'
                    ]

join_field_names = ['Residential_Units']

# List of fields to be joined using SUM
fields_to_sum = ['Residential_Units']

# Create a field map object
field_mappings = arcpy.FieldMappings()

# Get the field info for the target feature class
target_field_info = arcpy.ListFields(target_feature_class)

# Add fields from the target feature class to the field mappings
for field in target_field_info:
    if field.name in input_field_names:
        input_field_map = arcpy.FieldMap()
        input_field_map.addInputField(target_feature_class, field.name)
        # Set the merge rule to SUM for selected fields
        if field.name in fields_to_sum:
            input_field_map.mergeRule = "SUM"
        
        field_mappings.addFieldMap(input_field_map)

# Get the field info for the join feature class
join_field_info = arcpy.ListFields(join_feature_class)

# Add fields from the join feature class to the field mappings
for field in join_field_info:
    if field.name in join_field_names:
        join_field_map = arcpy.FieldMap()
        join_field_map.addInputField(join_feature_class, field.name)
        # Set the merge rule to SUM for selected fields
        if field.name in fields_to_sum:
            join_field_map.mergeRule = "SUM"
        # add join fields to field mappings
        field_mappings.addFieldMap(join_field_map)

# Perform the spatial join using the specified field mappings
arcpy.analysis.SpatialJoin(
                    target_feature_class, 
                    join_feature_class, 
                    output_feature_class,
                    join_operation="JOIN_ONE_TO_ONE",
                    join_type="KEEP_ALL",
                    field_mapping = field_mappings,
                    match_option="CONTAINS",
                    search_radius=None,
                    distance_field_name="")


<Result 'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012_ParcelDev2022'>

In [65]:
# Set local variables
in_table = os.path.join(workspace,"SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012_ParcelDev2022") 
field = "Residential_Units"
new_field_name = "Residential_Units_2022"
new_field_alias = "Residential Units 2022"
field_type = "LONG"

# Alter the properties of a non nullable, short data type field to become a text field
arcpy.management.AlterField(in_table,
                            field,
                            new_field_name,
                            new_field_alias,
                            field_type)

<Result 'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012_ParcelDev2022'>

In [66]:
spJoin = os.path.join(workspace, "SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012_ParcelDev2022")
arcpy.AddField_management(spJoin, 'Population_PercentChange', "DOUBLE")
arcpy.AddField_management(spJoin, 'Housing_PercentChange', "DOUBLE")
arcpy.AddField_management(spJoin, 'Occupied_PercentChange', "DOUBLE")
arcpy.AddField_management(spJoin, 'Vacant_PercentChange', "DOUBLE")
arcpy.AddField_management(spJoin, 'Seasonal_PercentChange', "DOUBLE")
arcpy.AddField_management(spJoin, 'Development_PercentChange', "DOUBLE")

<Result 'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\SpatialJoin_Blocks2020_Blocks2010_ParcelDev2012_ParcelDev2022'>

In [67]:
# fields
field_names = [f.name for f in arcpy.ListFields(spJoin)]
print(field_names)

['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'GEOGRAPHY', 'GEOID', 'GlobalID', 'NEIGHBORHOOD', 'STATE', 'TRPAID', 'YEAR', 'Block_Population_2020', 'Block_HousingUnits_2020', 'Block_Occupied_2020', 'Block_Vacant_2020', 'Block_Seasonal_2020', 'Block_Population_2010', 'Block_HousingUnits_2010', 'Block_Occupied_2010', 'Block_Vacant_2010', 'Block_Seasonal_2010', 'Residential_Units_2012', 'Residential_Units_2022', 'Shape_Length', 'Shape_Area', 'Population_PercentChange', 'Housing_PercentChange', 'Occupied_PercentChange', 'Vacant_PercentChange', 'Seasonal_PercentChange', 'Development_PercentChange']


In [1]:
# Start an edit session
edit = arcpy.da.Editor(workspace)
edit.startEditing(False, True)

# Define the fields
field1 = "Block_Population_2020"
field2 = "Block_Population_2010"
percent_change_field = "Population_PercentChange"

# Calculate percent change using an update cursor
with arcpy.da.UpdateCursor(spJoin, [field1, field2, percent_change_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[0] != 0:
            percent_change = ((row[1] - row[0]) / row[0]) * 100
            row[2] = percent_change
            cursor.updateRow(row)


# Define the fields
field1 = "Block_HousingUnits_2020"
field2 = "Block_HousingUnits_2010"
percent_change_field = "Housing_PercentChange"

# Calculate percent change using an update cursor
with arcpy.da.UpdateCursor(spJoin, [field1, field2, percent_change_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[0] != 0:
            percent_change = ((row[1] - row[0]) / row[0]) * 100
            row[2] = percent_change
            cursor.updateRow(row)

# Define the fields
field1 = "Block_Occupied_2020"
field2 = "Block_Occupied_2010"
percent_change_field = "Occupied_PercentChange"

# Calculate percent change using an update cursor
with arcpy.da.UpdateCursor(spJoin, [field1, field2, percent_change_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[0] != 0:
            percent_change = ((row[1] - row[0]) / row[0]) * 100
            row[2] = percent_change
            cursor.updateRow(row)

# Define the fields
field1 = "Block_Vacant_2020"
field2 = "Block_Vacant_2010"
percent_change_field = "Vacant_PercentChange"

# Calculate percent change using an update cursor
with arcpy.da.UpdateCursor(spJoin, [field1, field2, percent_change_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[0] != 0:
            percent_change = ((row[1] - row[0]) / row[0]) * 100
            row[2] = percent_change
            cursor.updateRow(row)

# Define the fields
field1 = "Block_Seasonal_2020"
field2 = "Block_Seasonal_2010"
percent_change_field = "Seasonal_PercentChange"


# Calculate percent change using an update cursor
with arcpy.da.UpdateCursor(spJoin, [field1, field2, percent_change_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[0] != 0:
            percent_change = ((row[1] - row[0]) / row[0]) * 100
            row[2] = percent_change
            cursor.updateRow(row)

# Define the fields
field1 = "Residential_Units_2022"
field2 = "Residential_Units_2012"
percent_change_field = "Development_PercentChange"


# Calculate percent change using an update cursor
with arcpy.da.UpdateCursor(spJoin, [field1, field2, percent_change_field]) as cursor:
    for row in cursor:
        if row[0] is not None and row[1] is not None and row[0] != 0:
            percent_change = ((row[1] - row[0]) / row[0]) * 100
            row[2] = percent_change
            cursor.updateRow(row)

            
# Stop the edit session
edit.stopEditing(True)

NameError: name 'arcpy' is not defined

# Aggregate TDC data for Block Groups

## Download Data

In [6]:
def get_census_data(featureset):
    service_id = {
        'raw_data':'28',
        'summaries':'18'
    }

    service_number = service_id.get(featureset) 
    service_url = 'https://maps.trpa.org/server/rest/services/Demographics/FeatureServer/'+service_number

    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features

    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])

    return all_data


def categorize_values(census_df, category_csv, category_column, grouping_prefix):
    categories = pd.read_csv(category_csv)    
    census_df['value'] = census_df['value'].astype(float)
    joined_data = census_df.merge(categories, on = 'variable_code', how = 'left')
    joined_data.sort_values(by='variable_code', inplace=True)
    #This will get rid of any extra columns in the category_csv
    group_columns = [column for column in census_df if column not in ['value', 'variable_code', 'variable_name', 'MarginOfError','OBJECTID']]
    group_columns.append(category_column)
    #grouped_data = joined_data.groupby(group_columns, as_index=False)['value'].sum()    
    print(group_columns)
    grouped_data = joined_data.groupby(group_columns, as_index=False, dropna=False).agg({'value':'sum',
                                                                           'variable_code':lambda x: grouping_prefix +  ', '.join(x)})
    
    #Need to return this formatted for appending to the table - need to get locations of variable_code and variable name, 
    #add them in as columns in those locations and then populate them with category column nanme
    var_code_col_location = census_df.columns.get_loc('variable_code')
    var_name_col_location = census_df.columns.get_loc('variable_name')
    var_moe_col_location = census_df.columns.get_loc('MarginOfError')
    grouped_data.insert(var_moe_col_location, 'MarginOfError', '')
    #grouped_data.insert(var_code_col_location, 'variable_code','Grouped Value')
    grouped_data.insert(var_name_col_location, 'variable_name','')
    #grouped_data['variable_code'] = grouped_data['variable_code'] +  '_Grouped'
    grouped_data['variable_name'] = grouped_data[category_column]
    grouped_data['dataset']= grouping_prefix + grouped_data['dataset']
    grouped_data['variable_category']= grouping_prefix +  grouped_data['variable_category'] 
    columns_to_keep = [column for column in census_df if column not in ['OBJECTID']]
    grouped_data= grouped_data[columns_to_keep]
    return grouped_data

In [16]:
census_data = get_census_data('raw_data')
census_data = census_data.loc[census_data['county']!='510']
county_zone = {
    '061':'North Lake',
    '031':'North Lake',
    '005': 'South Lake',
    '017':'South Lake'
}
census_data['Zone'] = census_data['county'].map(county_zone)

In [19]:
TDC_Variables = pd.read_csv("Census_Category_Lists\TDC_categories.csv")

In [36]:
block_group_data = census_data.loc[(census_data['sample_level']=='block group')&(census_data['year_sample']==2021)]
# Map Bipoc, seniors, youth, 0 vehicle households (fillna for no categories)
tdc_data = block_group_data.loc[block_group_data['variable_code'].isin(TDC_Variables['variable_code'])]

sort_order = ['TRPAID', 'county', 'tract', 'Total_Population',
'Total_Households',
'Total_Disability_Population',
'Age_Under_18',
'Age_18_to_65',
'Age_65_and_Over',
'With_Disability',
'Vehicle_Available_0',
'Vehicle_Available_1',
'Vehicle_Available_2',
'Vehicle_Available_3',
'Vehicle_Available_4',
'Vehicle_Available_5',
'White_Alone',
'BIPOC'
]

# Grab the category assigning function
tdc_grouped_data = categorize_values(tdc_data, "Census_Category_Lists\TDC_categories.csv", 'Broad Category', "")

tdc_flat = tdc_grouped_data.pivot(index=['TRPAID', 'county', 'tract'], columns='variable_name', values='value')
tdc_flat = tdc_flat.reset_index()
tdc_flat = tdc_flat[sort_order]

['GEO_ID', 'state', 'county', 'tract', 'block_group', 'variable_category', 'year_sample', 'sample_level', 'dataset', 'census_geom_year', 'TRPAID', 'NEIGHBORHOOD', 'Data_Scale', 'Zone', 'Broad Category']


C:\Users\amcclary\AppData\Local\Temp\ipykernel_52904\2306401841.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_df['value'] = census_df['value'].astype(float)


In [37]:
feature_layer_item = gis.content.search(query="Demographics", item_type="Feature Layer")[0]

# feature sub layer name to get
sublayer_name = "Tahoe Census Geography"
# get census table from the feature service
table_name = "Census Data"

# Query the sublayer by name
sublayer = None
for layer in feature_layer_item.layers:
    if layer.properties.name == sublayer_name:
        sublayer = layer
        break

sdfCensus = pd.DataFrame.spatial.from_layer(sublayer)

In [42]:
merged_df = pd.merge(sdfCensus, tdc_flat, on='TRPAID', how='inner')
columns_drop=['GlobalID', 'YEAR', 'created_date', 'created_user', 'last_edited_date', 'last_edited_user', 'Shape__Area', 'Shape__Length']
merged_df = merged_df.drop(columns=columns_drop)

In [43]:
## Export spatial dataframes to feature class to use in Spatial join
merged_df.spatial.to_featureclass(os.path.join(workspace, "Tahoe_BlockGroup_2021_TDC_Values"), sanitize_columns=False)

'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\Tahoe_BlockGroup_2021_TDC_Values'

# Create Feature Layers

### List of layers to create
#### Blocks
* 2020 population, housing
* 2010 population, housing
* percent change/bivariate

#### Block Group/Tracts Maps
* population change
* housing vacant/occupied/seasonal change
* race dot density
* choropleth of population, housing units, median income

In [9]:
# Search for the feature service by keyword
feature_layer_item = gis.content.search(query="Demographics", item_type="Feature Layer")[0]

# feature sub layer name to get
sublayer_name = "Tahoe Census Geography"
# get census table from the feature service
table_name = "Census Data"

# Query the sublayer by name
sublayer = None
for layer in feature_layer_item.layers:
    if layer.properties.name == sublayer_name:
        sublayer = layer
        break

# Query the table by name
subtable = None
for table in feature_layer_item.tables:
    if table.properties.name == table_name:
        subtable = table
        break
        
# create a DataFrame from the table
sdfCensus = pd.DataFrame.spatial.from_layer(sublayer)
dfCensus  = pd.DataFrame.spatial.from_layer(subtable)

# create data frames from filter for block and year
sdfTract2020 = sdfCensus[(sdfCensus['GEOGRAPHY'] == 'Tract') & (sdfCensus['YEAR'] == 2020)]
sdfTract2010 = sdfCensus[(sdfCensus['GEOGRAPHY'] == 'Tract') & (sdfCensus['YEAR'] == 2010)]

# Define the filter conditions for each field in the table
conditionTract      = dfCensus['sample_level']  == 'tract'
conditionLevel      = dfCensus['dataset']       == 'dec/dp'
condition2010       = dfCensus['year_sample']   == 2010
condition2020       = dfCensus['year_sample']   == 2020
conditionPopulation = dfCensus['variable_name'] == 'Total Population'
conditionHousing    = dfCensus['variable_name'] == 'Total Housing Units'
conditionOccupied   = dfCensus['variable_name'] == 'Total Housing Units: Occupied'
conditionVacant     = dfCensus['variable_name'] == 'Total Housing Units: Vacant'
conditionSeasonal   = dfCensus['variable_name'] == 'Vacant Housing Units: Seasonal, recreational, or occasional use'

# filter to create new dfs by variable name in the table
dfTractPop2010           =  dfCensus.loc[conditionTract & condition2010 & conditionPopulation].copy()
dfTractUnits2010         =  dfCensus.loc[conditionTract & condition2010 & conditionHousing].copy()
dfTractUnitsOccupied2010 =  dfCensus.loc[conditionTract & condition2010 & conditionOccupied].copy()
dfTractUnitsVacant2010   =  dfCensus.loc[conditionTract & condition2010 & conditionVacant].copy()
dfTractUnitsSeasonal2010 =  dfCensus.loc[conditionTract & condition2010 & conditionSeasonal].copy()

# create 2020 data frames 
dfTractPop2020           =  dfCensus.loc[conditionTract & conditionLevel & condition2020 & conditionPopulation].copy()
dfTractUnits2020         =  dfCensus.loc[conditionTract & conditionLevel & condition2020 & conditionHousing].copy()
dfTractUnitsOccupied2020 =  dfCensus.loc[conditionTract & conditionLevel & condition2020 & conditionOccupied].copy()
dfTractUnitsVacant2020   =  dfCensus.loc[conditionTract & conditionLevel & condition2020 & conditionVacant].copy()
dfTractUnitsSeasonal2020 =  dfCensus.loc[conditionTract & conditionLevel & condition2020 & conditionSeasonal].copy()

# list of dataframes used to name the value fields
df_list = [
    dfTractPop2010,          
    dfTractUnits2010,         
    dfTractUnitsOccupied2010, 
    dfTractUnitsVacant2010,  
    dfTractUnitsSeasonal2010,
    dfTractPop2020,           
    dfTractUnits2020,         
    dfTractUnitsOccupied2020, 
    dfTractUnitsVacant2020,
    dfTractUnitsSeasonal2020
]

# specify fields to keep in each dataframe from the table queries
fields_to_keep = ['TRPAID', 'value']

# Loop through each DataFrame and drop the specified fields and 
# rename the value fields with dataframe names as the prefix
for dataframe in df_list:
    # drop columns not in list
    dataframe.drop(columns=[col for col in dataframe.columns if col not in fields_to_keep], inplace=True)  
    # Exclude the field name you want to skip
    field_to_exclude = "TRPAID"
    # keep neccessary columns
    included_columns = [col for col in dataframe.columns if col != field_to_exclude]
    # get the dataframe name as a string
    df_name = [name for name in globals() if globals()[name] is dataframe][0]
    dataframe['TRPAID'].astype(str)
    # Add DataFrame name as a prefix to included column names
    new_columns = [f"{df_name}_{col}" for col in included_columns]
    # add the new name back to the column list
    dataframe.columns =  ["TRPAID"]+new_columns
    
# numeric_columns = []

# for column in sdfTract2020.columns:
#     if pd.api.types.is_numeric_dtype(sdfTract2020[column]):
#         numeric_columns.append(column)
        
# # Fill specified numeric columns with zeros
# sdfTract2020[numeric_columns] = sdfTract2020.loc[numeric_columns].fillna(0)

# numeric_columns = []

# for column in sdfTract2010.columns:
#     if pd.api.types.is_numeric_dtype(sdfTract2010[column]):
#         numeric_columns.append(column)
        
# # # Fill specified numeric columns with zeros
# sdfTract2010[numeric_columns] = sdfTract2010.loc[numeric_columns].fillna(0)

In [11]:
dfTractUnits2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 23435 to 23466
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   TRPAID                  32 non-null     string 
 1   dfTractUnits2020_value  32 non-null     Float64
dtypes: Float64(1), string(1)
memory usage: 800.0 bytes


In [6]:
sdfTract2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 512 entries, 0 to 511
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   GEOGRAPHY                512 non-null    string  
 1   GEOID                    512 non-null    Float64 
 2   GlobalID                 512 non-null    string  
 3   NEIGHBORHOOD             496 non-null    string  
 4   OBJECTID                 512 non-null    Int64   
 5   SHAPE                    512 non-null    geometry
 6   STATE                    512 non-null    string  
 7   TRPAID                   512 non-null    string  
 8   YEAR                     512 non-null    Int32   
 9   Tract_Population_2020    512 non-null    Float64 
 10  Tract_HousingUnits_2020  512 non-null    Float64 
 11  Tract_Occupied_2020      512 non-null    Float64 
 12  Tract_Vacant_2020        512 non-null    Float64 
 13  Tract_Seasonal_2020      512 non-null    Float64 
dtypes: Float64

In [12]:
# get list of dataframes to merge with 2010 spatial dataframe
df_list_2010 = [
    dfTractPop2010,          
    dfTractUnits2010,         
    dfTractUnitsOccupied2010, 
    dfTractUnitsVacant2010,  
    dfTractUnitsSeasonal2010
]
# get list of dataframes to merge with 2020 spatial dataframe
df_list_2020= [
    dfTractPop2020,           
    dfTractUnits2020,         
    dfTractUnitsOccupied2020, 
    dfTractUnitsVacant2020,
    dfTractUnitsSeasonal2020
]
# merge dataframes to spatial dataframe
for df in df_list_2010:
    sdfTract2010 = sdfTract2010.merge(df, on='TRPAID', how="left")
# so it for 2020
for df in df_list_2020:
    sdfTract2020 = sdfTract2020.merge(df, on='TRPAID', how="left")
    
# set new field values
sdfTract2010['Tract_Population_2010']   = sdfTract2010['dfTractPop2010_value']
sdfTract2010['Tract_HousingUnits_2010'] = sdfTract2010['dfTractUnits2010_value']
sdfTract2010['Tract_Occupied_2010']     = sdfTract2010['dfTractUnitsOccupied2010_value']
sdfTract2010['Tract_Vacant_2010']       = sdfTract2010['dfTractUnitsVacant2010_value']
sdfTract2010['Tract_Seasonal_2010']     = sdfTract2010['dfTractUnitsSeasonal2010_value']

sdfTract2020['Tract_Population_2020']   = sdfTract2020['dfTractPop2020_value']
sdfTract2020['Tract_HousingUnits_2020'] = sdfTract2020['dfTractUnits2020_value']
sdfTract2020['Tract_Occupied_2020']     = sdfTract2020['dfTractUnitsOccupied2020_value']
sdfTract2020['Tract_Vacant_2020']       = sdfTract2020['dfTractUnitsVacant2020_value']
sdfTract2020['Tract_Seasonal_2020']     = sdfTract2020['dfTractUnitsSeasonal2020_value']


# Drop columns not in the list
columns_to_drop_2010 = ['Shape__Area', 'Shape__Length', 'created_date', 'created_user', 
                        'last_edited_date', 'last_edited_user', 
                        'dfTractPop2010_value', 'dfTractUnits2010_value', 
                        'dfTractUnitsOccupied2010_value', 'dfTractUnitsVacant2010_value', 
                        'dfTractUnitsSeasonal2010_value']

# Drop columns not in the list
columns_to_drop_2020 = ['Shape__Area', 'Shape__Length', 'created_date', 'created_user', 
                        'last_edited_date', 'last_edited_user', 
                        'dfTractPop2020_value', 'dfTractUnits2020_value', 
                        'dfTractUnitsOccupied2020_value', 'dfTractUnitsVacant2020_value', 
                        'dfTractUnitsSeasonal2020_value']

# drop
sdfTract2010.drop(columns=columns_to_drop_2010, inplace=True)

# drop in place
sdfTract2020.drop(columns=columns_to_drop_2020, inplace=True)

## Export spatial dataframes to feature class to use in Spatial join
sdfTract2010.spatial.to_featureclass(os.path.join(workspace, "Tahoe_Tract_2010_Values"), sanitize_columns=False)

# Export spatial dataframes to feature class to use in Spatial join
sdfTract2020.spatial.to_featureclass(os.path.join(workspace, "Tahoe_Tract_2020_Values"), sanitize_columns=False)

'F:\\GIS\\PROJECTS\\ResearchAnalysis\\Demographics\\Workspace.gdb\\Tahoe_Tract_2020_Values'